In [73]:
pip install polyline

Note: you may need to restart the kernel to use updated packages.


In [74]:
pip install haversine 

Note: you may need to restart the kernel to use updated packages.


In [75]:
import pandas as pd
import numpy as np
import os

In [104]:
import requests

from geopy.distance import geodesic

from datetime import datetime, timedelta, date

import polyline
from haversine import haversine, Unit

Import Customer Master

In [77]:
file_path = r'C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\Customer Master.xlsx'
customer_df = pd.read_excel(file_path, sheet_name='Customers Transit2025', dtype={'Zip Code': str})

customer_df.head(5)

,Dest. Customer Name,Destination City,Destination State,Order Frequency Per Month,Preferred ship day,Transit time,Zip Code,Customer Code,Customer Short Name,Region,Division,dropoff_lat,dropoff_lng,route distance mile,Customer Status
0,High Grade Beverage - Monmouth Junction,Monmouth Junction,NJ,1.0,flex,1,08852,C0001,High Grade Bev - Monmouth,Northeast,Middle Atlantic,40.38978,-74.54461,358,Active
1,High Grade Beverage - Randolph,Randolph,NJ,2.0,flex,1,07869,C0002,High Grade Bev - Randolph,Northeast,Middle Atlantic,40.84528,-74.58070,349,Active
2,Konrad Beverage,Deptford,NJ,0.5,flex,1,08096,C0003,Konrad Beverage,Northeast,Middle Atlantic,39.82921,-75.12871,409,Active
3,Northern Eagle Beverage Company,Carlstadt,NJ,2.0,flex,1,07072,C0004,Northern Eagle,Northeast,Middle Atlantic,40.82472,-74.06126,317,Active
4,"Ritchie & Page Distributing Co., Inc.",Robbinsville,NJ,2.0,flex,1,08691,C0005,Ritchie & Page,Northeast,Middle Atlantic,40.20953,-74.59919,374,Active


Import Waypoints

In [78]:
file_path = r'C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\waypoints_detail_df.xlsx'
waypoints_detail_df = pd.read_excel(file_path, dtype={'Pick Zip': str, 'Drop Zip': str, 'Waypoint Zip': str})

waypoints_detail_df.head()

,Customer Code,Customer Short Name,Pick Zip,Drop Zip,Sequence,Waypoint Zip,Waypoint Lat,Waypoint Lng,Duration_s_pessimistic,Location Type,%Duration_pessimistic
0,C0001,High Grade Bev - Monmouth,04103,08852,0,04103,43.689205,-70.272004,18,Start,0.000862
1,C0001,High Grade Bev - Monmouth,04103,08852,1,04103,43.688767,-70.272323,26,Checkpoint,0.001245
2,C0001,High Grade Bev - Monmouth,04103,08852,2,04103,43.688104,-70.271100,15,Checkpoint,0.000718
3,C0001,High Grade Bev - Monmouth,04103,08852,3,04103,43.687620,-70.271636,176,Checkpoint,0.008430
4,C0001,High Grade Bev - Monmouth,04103,08852,4,04103,43.680229,-70.257689,476,Checkpoint,0.022798


Polylines

In [79]:
# Function to encode polyline for each customer
def encode_polyline(group):
    waypoints = list(zip(group["Waypoint Lat"], group["Waypoint Lng"]))
    return polyline.encode(waypoints)

# Group by
encoded_routes = waypoints_detail_df.groupby("Customer Code").apply(encode_polyline)

# Convert the series to DataFrame and reset index
encoded_routes_df = encoded_routes.reset_index(name="Encoded_Polyline")

# Merge the encoded polyline back into the original DataFrame
customer_df = customer_df.merge(
    encoded_routes_df, 
    on="Customer Code", 
    how="left"
)

In [80]:
customer_df.head(5)

,Dest. Customer Name,Destination City,Destination State,Order Frequency Per Month,Preferred ship day,Transit time,Zip Code,Customer Code,Customer Short Name,Region,Division,dropoff_lat,dropoff_lng,route distance mile,Customer Status,Encoded_Polyline
0,High Grade Beverage - Monmouth Junction,Monmouth Junction,NJ,1.0,flex,1,08852,C0001,High Grade Bev - Monmouth,Northeast,Middle Atlantic,40.38978,-74.54461,358,Active,q`tiG~~{kLvA~@dCsF~AjBdm@evAr{J`pR|g~Bze`BzzSz...
1,High Grade Beverage - Randolph,Randolph,NJ,2.0,flex,1,07869,C0002,High Grade Bev - Randolph,Northeast,Middle Atlantic,40.84528,-74.58070,349,Active,q`tiG~~{kLvA~@dCsF~AjBdm@evAr{J`pR|g~Bze`BzzSz...
2,Konrad Beverage,Deptford,NJ,0.5,flex,1,08096,C0003,Konrad Beverage,Northeast,Middle Atlantic,39.82921,-75.12871,409,Active,q`tiG~~{kLvA~@dCsF~AjBdm@evAr{J`pR|g~Bze`BzzSz...
3,Northern Eagle Beverage Company,Carlstadt,NJ,2.0,flex,1,07072,C0004,Northern Eagle,Northeast,Middle Atlantic,40.82472,-74.06126,317,Active,q`tiG~~{kLvA~@dCsF~AjBdm@evAr{J`pR|g~Bze`BzzSz...
4,"Ritchie & Page Distributing Co., Inc.",Robbinsville,NJ,2.0,flex,1,08691,C0005,Ritchie & Page,Northeast,Middle Atlantic,40.20953,-74.59919,374,Active,q`tiG~~{kLvA~@dCsF~AjBdm@evAr{J`pR|g~Bze`BzzSz...


In [81]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Dest. Customer Name        58 non-null     object 
 1   Destination City           58 non-null     object 
 2   Destination State          58 non-null     object 
 3   Order Frequency Per Month  53 non-null     float64
 4   Preferred ship day         48 non-null     object 
 5   Transit time               58 non-null     int64  
 6   Zip Code                   58 non-null     object 
 7   Customer Code              58 non-null     object 
 8   Customer Short Name        58 non-null     object 
 9   Region                     58 non-null     object 
 10  Division                   58 non-null     object 
 11  dropoff_lat                58 non-null     float64
 12  dropoff_lng                58 non-null     float64
 13  route distance mile        58 non-null     int64  
 

In [82]:
# Step 1: Decode Polyline for Each Route
def decode_route(encoded_poly):
    """Decode polyline into a list of (lat, lng) tuples."""
    if isinstance(encoded_poly, str):  # Ensure it's a valid string
        return polyline.decode(encoded_poly)
    return []  # Return an empty list for missing values

# Apply the function safely
customer_df["Route_Points"] = customer_df["Encoded_Polyline"].apply(decode_route)

In [84]:
# Generate csv, #export csv
# Specify the full path and file name
file_path = r"C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\customer_df.xlsx"

# Export DataFrame to Excel
customer_df.to_excel(file_path, index=False)

In [ ]:
# # Step 2: Find Nearby Customers Along the Route
# def find_nearby_customers_by_distance(route_points, customers, primary_load_number):
#     """Find customers near a shipping route, calculate distances, and return a DataFrame linked to a Primary Load Number."""
    
#     if not route_points:  # Ensure route_points is not empty
#         return pd.DataFrame(columns=["Primary Load Number", "Zip Code", "Destination City", "Distance"])

#     data = []  # List to store customer records
    
#     for _, row in customers.iterrows():
#         customer_loc = (row["dropoff_lat"], row["dropoff_lng"])  # Get customer coordinates

#         for route_point in route_points:
#             distance = haversine(customer_loc, route_point, unit=Unit.MILES)

#             if distance <= 100:  # Only keep customers within 100 miles
#                 data.append({
#                     "Primary Load Number": primary_load_number,
#                     "Zip Code": row["Zip Code"],  # FIXED COLUMN NAME
#                     "Destination City": row["Destination City"],  # FIXED COLUMN NAME
#                     "Distance": round(distance, 2)
#                 })
                

#     # Convert list to DataFrame
#     return pd.DataFrame(data)

In [117]:
# Step 2: Find Nearby Customers Along the Route
def find_nearby_customers(customer_df, max_distance=100, active_status="Active"):
    """
    Finds nearby customers along each customer's route within a specified max_distance.

    Args:
        customer_df: Pandas DataFrame with customer data, including 'Customer Code', 'dropoff_lat', 'dropoff_lng', and 'Route_Points'.
        max_distance: The maximum distance (in miles) to consider a customer "nearby".

    Returns:
        A DataFrame with customer codes and nearby customer details.
    """
    # Filter only active customers
    active_customers_df = customer_df[customer_df["Customer Status"] == active_status]

    nearby_customers = []  # List to store nearby customers

    for i, row1 in active_customers_df.iterrows():  # Iterate through each customer (customer1)
        customer1_code = row1['Customer Code']
        customer1_lat = row1['dropoff_lat']
        customer1_lng = row1['dropoff_lng']
        route_points1 = row1['Route_Points']

        if not route_points1:  # Skip if route points are missing for customer1
            continue

        # Convert route points to separate lat/lng arrays for customer1
        route_lats1 = np.array([p[0] for p in route_points1])
        route_lngs1 = np.array([p[1] for p in route_points1])

        # Compare customer1's route points to customer2's dropoff lat/lng
        for j, row2 in active_customers_df.iterrows():  # Compare customer1 with customer2
            if i == j:  # Skip comparing the customer with itself
                continue

            customer2_code = row2['Customer Code']
            customer2_name = row2['Customer Short Name']
            customer2_lat = row2['dropoff_lat']
            customer2_lng = row2['dropoff_lng']

            # Compute the distances from each route point of customer1 to customer2's drop-off location
            distances = np.array([geodesic((lat1, lng1), (customer2_lat, customer2_lng)).miles for lat1, lng1 in zip(route_lats1, route_lngs1)])

            min_distance = min(distances)
            if min_distance <= max_distance:
                # Add nearby customer to the list
                nearby_customers.append({
                    "Customer Code": customer1_code,
                    "Nearby Customer Code": customer2_code,
                    "Nearby Customer Name": customer2_name,
                    "Nearby Customer lat": customer2_lat,
                    "Nearby Customer lng": customer2_lng,
                    "Distance (miles)": round(min_distance, 2)
                })

    # Return the results as a DataFrame
    return pd.DataFrame(nearby_customers)

In [118]:
# Apply the function
nearby_customers_df = find_nearby_customers(customer_df, max_distance=100)

In [119]:
nearby_customers_df.head(5)

,Customer Code,Nearby Customer Code,Nearby Customer Name,Nearby Customer lat,Nearby Customer lng,Distance (miles)
0,C0001,C0002,High Grade Bev - Randolph,40.84528,-74.58070,24.94
1,C0001,C0003,Konrad Beverage,39.82921,-75.12871,49.53
2,C0001,C0004,Northern Eagle,40.82472,-74.06126,2.36
3,C0001,C0005,Ritchie & Page,40.20953,-74.59919,12.77
4,C0001,C0012,Northeast Bev,41.28270,-73.02720,21.35


In [120]:
# Generate csv, #export csv
# Specify the full path and file name
file_path = r"C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\nearby_customers_df.xlsx"

# Export DataFrame to Excel
nearby_customers_df.to_excel(file_path, index=False)

waypoints + nearby customers

In [121]:
def add_nearby_customers_to_waypoints(customer_df, waypoints_df, nearby_customers_df):
    """Add nearby customers as waypoints to the waypoints DataFrame."""
    
    new_waypoints = []  # Store new waypoint entries as a list
    
    for _, row in customer_df.iterrows():
        customer_code = row["Customer Code"]
        DropZip = row["Zip Code"]
        
        # Find nearby customers for the given Customer Code
        nearby_customers = nearby_customers_df[nearby_customers_df["Customer Code"] == customer_code]
        
        for _, nearby in nearby_customers.iterrows():
            new_entry = {
                "Customer Code": customer_code,
                "Customer Short Name": row["Customer Short Name"],
                "Pick Zip": '04103',
                "Drop Zip": DropZip,
                "Sequence": None,  # Since it's a nearby customer
                "Waypoint Zip": None,  # No zip code from route
                "Waypoint Lat": nearby["Nearby Customer lat"],
                "Waypoint Lng": nearby["Nearby Customer lng"],
                "Duration_s_pessimistic": None,  # NA since it's not a calculated waypoint
                "%Duration_pessimistic": None,  # NA since no duration
                "Location Type": "Nearby Customer",
                "Nearby Customer Code": nearby["Nearby Customer Code"],
                "Nearby Customer Name": nearby["Nearby Customer Name"],
                "Distance (miles)": nearby["Distance (miles)"]
            }
            new_waypoints.append(new_entry)  

    # Convert new waypoints to DataFrame **only if list is not empty**
    if new_waypoints:
        new_waypoints_df = pd.DataFrame(new_waypoints)

        # Ensure both DataFrames are non-empty before concatenation
        if not waypoints_df.empty:
            waypoints_df = pd.concat([waypoints_df, new_waypoints_df], ignore_index=True)
        else:
            waypoints_df = new_waypoints_df  # If waypoints_df was empty, just use the new data
    elif waypoints_df.empty:
        waypoints_df = pd.DataFrame() # create an empty dataframe if both are empty.

    return waypoints_df


In [122]:
waypoints_n_nearby_df = []

waypoints_n_nearby_df = add_nearby_customers_to_waypoints(customer_df, waypoints_detail_df, nearby_customers_df)


C:\Users\sumat\AppData\Local\Temp\ipykernel_14488\1531708906.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  waypoints_df = pd.concat([waypoints_df, new_waypoints_df], ignore_index=True)


In [123]:
# Generate csv, #export csv
# Specify the full path and file name
file_path = r"C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\waypoints_n_nearby_df.xlsx"

# Export DataFrame to Excel
waypoints_n_nearby_df.to_excel(file_path, index=False)